In [ ]:
import folium
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

# Dados

In [ ]:
planilha = 'Data for capacitated vehicle routing problem.xlsx'

# Origem
df_origem = pd.read_excel(planilha, sheet_name = 'Origem')
origem = {'ORI-001': df_origem.get(['Latitude', 'Longitude']).values.tolist()[0]}
# print(f'Origem: {origem}')

# Custos dos caixeiros
df_veiculos = pd.read_excel(planilha, sheet_name = 'Veículos')
custos_caixeiros = dict(df_veiculos.get(['Placa', 'Custo / Km']).values)
custos_caixeiros = {str(c): float(custos_caixeiros[c]) for c in custos_caixeiros}
# print(f'Custos dos caixeiros: {custos_caixeiros}')

# Volumes dos caixeiros
volumes_caixeiros = dict(df_veiculos.get(['Placa', 'Cubagem (m³)']).values)
volumes_caixeiros = {str(c): float(volumes_caixeiros[c]) for c in volumes_caixeiros}
# print(f'Volumes dos caixeiros: {volumes_caixeiros}'')

# Pesos dos caixeiros
pesos_caixeiros = dict(df_veiculos.get(['Placa', 'Lotação (kg)']).values)
pesos_caixeiros = {str(c): float(pesos_caixeiros[c]) for c in pesos_caixeiros}
# print(f'Pesos dos caixeiros: {pesos_caixeiros}')

# Pontos de entrega
df_pedidos = pd.read_excel(planilha, sheet_name = 'Pedido')
entregas = sorted(list(set(df_pedidos.get('Destinatário'))))
lista_pontos = pd.read_excel( planilha, sheet_name = 'Destinatário', \
                              usecols = ['Código', 'Latitude', 'Longitude'] ).values.tolist()
pontos_entregas = {str(c): [float(l1), float(l2)] for c, l1, l2 in lista_pontos if c in entregas}
pontos = {**origem, **pontos_entregas}
# print(f'Pontos de entregas: {pontos_entregas}')
# print(f'Pontos: {pontos}')

# Volumes e pesos das entregas
df_produtos = pd.read_excel(planilha, sheet_name = 'Produto')

volumes_entregas = {'ORI-001': float(0)}
pesos_entregas = {'ORI-001': float(0)}
for e in entregas:
    volume = float(0)
    peso = float(0)
    for p in df_pedidos.loc[df_pedidos['Destinatário'] == e].values.tolist():
        prod = p[1]
        quan = float(p[3])
        corr = df_produtos.loc[df_produtos['Código'] == prod].values.tolist()
        volume += quan * float(corr[0][4])
        peso += quan * float(corr[0][3])
    volumes_entregas[e] = volume
    pesos_entregas[e] = peso
# print(f'Volumes das entregas: {volumes_entregas}')
# print(f'Pesos das entregas: {pesos_entregas}')

# Uma função auxiliar

In [ ]:
# Dados dois labels de pontos de armazenamento ou de entregas, essa função
# retorna a distância euclidiana entre os pontos correspondentes.

def dist(string1, string2):
    v = pontos[string1]
    w = pontos[string2]
    d = np.linalg.norm(np.array(v) - np.array(w))
    return float(d)

# Variáveis

In [ ]:
modelo = gp.Model()

# Lista de caixeiros
caixeiros = custos_caixeiros.keys()

# Variável binária que descreve se o trecho entre dois pontos de entrega é percorrido por dado caixeiro (1) ou não (0)
x = modelo.addVars(pontos.keys(), pontos.keys(), caixeiros, vtype = GRB.BINARY, name = 'x')

# Variável auxiliar usada para a eliminação de subrotas
u = modelo.addVars(pontos.keys(), vtype = GRB.INTEGER, name = 'u')

# Função objetivo

In [ ]:
# Objetivo: minimizar o custo total de todas as entregas
modelo.setObjective( gp.quicksum(dist(i, j) * custos_caixeiros[c] * x[i, j, c] for i in pontos.keys()
                                                                               for j in pontos.keys()
                                                                               for c in caixeiros),
                     sense = GRB.MINIMIZE )

# Restrições

In [ ]:
# Restrição: garante que todos os pontos de entrega sejam visitados exatamente uma vez
r1 = modelo.addConstrs( gp.quicksum( x[i, j, c] for i in pontos.keys() if i != j for c in caixeiros ) == 1 
                        for j in entregas )

# Restrição (conservação de fluxo): garante que todo caixeiro que entra em um ponto de entrega também sai 
r2 = modelo.addConstrs( gp.quicksum( (x[i, l, c] - x[l, i, c]) for i in pontos.keys() if i != l ) == 0
                        for l in pontos.keys() for c in caixeiros )

# Restrição: garante que a origem esteja presente no máximo umma vez em todos as rotas
r3 = modelo.addConstrs( gp.quicksum( x['ORI-001', j, c] for j in pontos.keys() ) == 1
                        for c in caixeiros )
r4 = modelo.addConstrs( gp.quicksum( x[i, 'ORI-001', c] for i in pontos.keys() ) == 1
                        for c in caixeiros )

# Restrição (eliminação de subrotas): garante que a rota de cada caixeiro siga a ordem dada pelo vetor u.
r5 = modelo.addConstrs( u[i] - u[j] + (len(pontos.keys())+1)*(x[i, j, c] - 1) <= -1 
                        for i in pontos.keys()
                        for j in entregas if j != i
                        for c in caixeiros )

# Restrição de empacotamento: o somatório dos volumes das entregas na rota de um caixeiro
# deve ser menor que o volume máximo que esse caixeiro é capaz de carregar
r6 = modelo.addConstrs(
         gp.quicksum( x[i, j, c] * volumes_entregas[j]  for i in entregas for j in pontos.keys() ) <= volumes_caixeiros[c]
         for c in caixeiros )

# Restrição de empacotamento: o somatório dos pesos das entregas na rota de um caixeiro
# deve ser menor que o peso máximo que esse caixeiro é capaz de carregar
r7 = modelo.addConstrs(
         gp.quicksum( x[i, j, c] * pesos_entregas[j]  for i in entregas for j in pontos.keys() ) <= pesos_caixeiros[c]
         for c in caixeiros )

# Resultados

In [ ]:
# Resolvendo o problema de otimização

modelo.optimize()

In [ ]:
# Construindo as trajetorias de cada caixeiro como um dicionário

trajetorias = dict()

for c in caixeiros:
    print(f'---------\n{c}:\n')

    trajetoria = ['ORI-001']
    i = 'ORI-001'
    j = [j for j in pontos.keys() if x[i, j, c].x != 0][0]
    trajetoria.append(j)
    while j != 'ORI-001':
        i = j
        j = [j for j in pontos.keys() if x[i, j, c].x != 0][0]
        trajetoria.append(j)
    trajetorias[c] = trajetoria
    print(trajetoria)
    
# print(trajetorias)

In [ ]:
# Vizualizando os resultados
    

colors = ['red', 'blue', 'green', 'purple', 'beige', 'orange', \
          'darkred', 'darkblue', 'darkgreen', 'darkpurple', \
          'lightred', 'lightblue', 'lightgreen', 'lightgray', 'pink', 'cadetblue']

mapa = folium.Map(location = origem['ORI-001'], zoom_start = 4)

for i, c in enumerate(trajetorias):
    cor = colors[i % len(colors)]
    traj = [pontos[l] for l in [*trajetorias[c], 'ORI-001']]
    for p in traj[1 : -1]:
        folium.Marker(p, icon = folium.Icon(color = cor, icon = 'location-dot')).add_to(mapa)
    folium.PolyLine(traj, color = cor).add_to(mapa)

folium.Marker(traj[0], icon = folium.Icon(color = 'black', icon = 'location-dot')).add_to(mapa)

display(mapa)